In [ ]:
import numpy as np
import os
import pandas
import json
from pathlib import Path
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import itertools
import ipywidgets as widgets
from IPython.display import display

In [ ]:
aggregate_data = pandas.read_pickle("../aggregate_data.pkl")
ground_truth_data = pandas.read_pickle("../ground_truth_data.pkl")
inferred_data = pandas.read_pickle("../inferred_data.pkl")
aggregate_data.columns
#inferred_data.columns

In [ ]:
pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")\
.groupby(["method_name", "system_name", "num_train_trajectories"])\
.agg({"relerr_l2" : "mean"})

In [ ]:
system_name = "spring"

In [ ]:
fig, ax = plt.subplots(2, 2)
fig.set_figwidth(30)
fig.set_figheight(20)

df = pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")\
.query("system_name == 'wave'")\
.groupby(["system_name", "method_name", "num_train_trajectories", "timestep_number"])\
.agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

df.loc[df["method_name"] == "mlp"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[0,0], logy=True)

df.loc[df["method_name"] == "hnn"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[1,0], logy=True)

df.loc[df["method_name"] == "srnn"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[0,1], logy=True)

df.loc[df["method_name"] == "knn-regressor"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[1,1], logy=True)

plt.suptitle("Spring")

def style_update(ax):
    ax.grid()
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    ax.legend(loc="lower right")
    ax.set_xlabel("Timestep number")
    ax.set_ylabel("Relative Error (mean over trajectories)")
    

[style_update(ax) for ax in itertools.chain(*ax)]


In [ ]:
def plot_system_performance(system_name):
    fig, ax = plt.subplots(1, 2)
    fig.suptitle(system_name)
    fig.set_figwidth(30)
    fig.set_figheight(10)

    df = pandas\
    .merge(aggregate_data, inferred_data, on="experiment_name")\
    .query("system_name == '{}'".format(system_name))\
    .groupby(["method_name", "system_name", "num_train_trajectories"])\
    .agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

    df\
    .pivot(index="method_name", columns=["num_train_trajectories"], values="mean_relerr_l2")\
    .plot(ax=ax[0], kind="bar", logy=True)

    df\
    .pivot(index="num_train_trajectories", columns=["method_name"], values="mean_relerr_l2")\
    .plot(ax=ax[1], kind="bar", logy=True)

    def style_update(ax):
        ax.grid()
        ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
        ax.legend(loc="upper center")
        ax.set_ylabel("Relative Error (mean over trajectories, timesteps)")

    [style_update(ax_) for ax_ in ax]
    plt.show()

plot_system_performance("spring")
plot_system_performance("wave")

In [ ]:
df = pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")
df = pandas.merge(df, ground_truth_data, on=["experiment_name", "trajectory_number", "timestep_number"])
run_df = df.loc[(df["system_name"] == "wave") &
                (df["method_name"] == "srnn") & 
                (df["network_depth"] == 3) & 
                (df["network_hidden_dim"] == 500) & 
                (df["num_train_trajectories"] == 1000)]
run_df.head()

In [ ]:
single_run = run_df.loc[(run_df["trajectory_number"] == 0)][["inferred_data", "ground_truth_data"]]
inferred_data_ = np.stack([entry[0] for entry in single_run.values])
ground_truth_data_ = np.stack([entry[1] for entry in single_run.values])
def plot_t(t):
    plt.figure(figsize=(12, 8))
    plt.plot(inferred_data_[t, ...], "*")
    plt.plot(ground_truth_data_[t, ...], ".-")
    plt.plot()

widgets.interact(plot_t, t=widgets.IntSlider(value=0, min=0, max=29))


In [ ]:
def plot_system_performance(system_name):
    fig, ax = plt.subplots(1)
    fig.suptitle(system_name)
    fig.set_figwidth(20)
    fig.set_figheight(10)

    df = pandas\
    .merge(aggregate_data, inferred_data, on="experiment_name")\
    .groupby(["method_name", "system_name"])\
    .agg({"inference_time" : "mean"}).add_prefix("mean_").reset_index()

    print(df)
    
    df\
    .pivot(index="method_name", columns=["system_name"], values="mean_inference_time")\
    .plot(ax=ax, kind="bar", logy=True)
        

    def style_update(ax):
        ax.grid()
        ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
        ax.legend(loc="upper center")
        ax.set_ylabel("Relative Error (mean over trajectories, timesteps)")

    style_update(ax)
    plt.show()
plot_system_performance(None)